# Purpose:
- To visualize mRuby2 z-stacks

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import tifffile as tf
from skimage import io
from PIL import Image
import sys
sys.path.append(r'C:\Users\jinho.kim\Github\lamf_ophys_analysis_dev')
from utils import zstack_reg

In [2]:
data_dir = Path(r'\\meso2acq\data\20240322_stacks')

In [11]:
zstack_fn_list = [data_dir / '718733_week2_redandgreen_visp_00001.tif',
                data_dir / '718732_week2_redandgreen_visp_00001.tif',
                data_dir / '712733_week3_redandgreen_visp_00001.tif',
                data_dir / '712732_week3_redandgreen_visp_00001.tif']


In [7]:
def get_zstack_reg(zstack, num_slices, ref_ind):
    plane_reg_imgs = np.zeros((num_slices, zstack.shape[1], zstack.shape[2]))
    shift_all_planes = []
    for plane_ind in range(num_slices):
        temp_imgs = zstack[plane_ind::num_slices]
        reg_img, shift_all = zstack_reg.average_reg_plane(temp_imgs)
        plane_reg_imgs[plane_ind] = reg_img
        shift_all_planes.append(shift_all)
    between_plane_reg_imgs, shift_all_between_planes = zstack_reg.reg_between_planes(np.array(plane_reg_imgs), ref_ind=ref_ind)
    return between_plane_reg_imgs, shift_all_planes, shift_all_between_planes

def get_zstack_reg_using_shifts(zstack, num_slices, shift_all_planes, shift_all_between_planes):
    plane_reg_imgs = np.zeros((num_slices, zstack.shape[1], zstack.shape[2]))
    for plane_ind in range(num_slices):
        temp_imgs = zstack[plane_ind::num_slices]
        reg_img = zstack_reg.average_reg_plane_using_shift_info(temp_imgs, shift_all_planes[plane_ind])
        plane_reg_imgs[plane_ind] = reg_img
    between_plane_reg_imgs = zstack_reg.reg_between_planes_using_shift_info(np.array(plane_reg_imgs), shift_all_between_planes)
    return between_plane_reg_imgs

In [12]:
# Takes about 100 min per stack
for zstack_fn in zstack_fn_list[2:]:
    si_metadata = zstack_reg.read_si_stack_metadata(zstack_fn)
    zstack = tf.imread(zstack_fn)
    assert zstack.shape[0] == si_metadata['num_slices'] * si_metadata['num_volumes']
    num_slices = si_metadata['num_slices']
    assert si_metadata['num_channels'] == 2

    ch = 1 # reference channel (red)
    zstack_ch = np.squeeze(zstack[:,ch,:,:])
    zstack_ch_reg, shift_all_planes, shift_all_between_planes = get_zstack_reg(zstack_ch, num_slices, 60)
    tf.imsave(zstack_fn.parent / f'{zstack_fn.stem}_ch{ch}_reg.tif', zstack_ch_reg)
    shift_info = {'shift_all_planes': shift_all_planes, 'shift_all_between_planes': shift_all_between_planes}
    np.save(zstack_fn.parent / f'{zstack_fn.stem}_ch{ch}_reg_shift_info.npy', shift_info)

    ch = 0 # target channel (green)
    zstack_ch = np.squeeze(zstack[:,ch,:,:])
    zstack_ch_reg = get_zstack_reg_using_shifts(zstack_ch, num_slices, shift_all_planes, shift_all_between_planes)
    tf.imsave(zstack_fn.parent / f'{zstack_fn.stem}_ch{ch}_reg_by_ch1.tif', zstack_ch_reg)

C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_9752\670178550.py:12: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tf.imsave(zstack_fn.parent / f'{zstack_fn.stem}_ch{ch}_reg.tif', zstack_ch_reg)
C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_9752\670178550.py:19: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tf.imsave(zstack_fn.parent / f'{zstack_fn.stem}_ch{ch}_reg_by_ch1.tif', zstack_ch_reg)


In [8]:
# it took 70 min to run self-registration for ch 1
# registration using previous result took 25 min
# so each round would take about 100 min
ch = 0 # target channel (green)
zstack_ch = np.squeeze(zstack[:,ch,:,:])
zstack_ch_reg = get_zstack_reg_using_shifts(zstack_ch, num_slices, shift_all_planes, shift_all_between_planes)
tf.imsave(zstack_fn.parent / f'{zstack_fn.stem}_ch{ch}_reg_by_ch1.tif', zstack_ch_reg)

C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_9752\1548072163.py:4: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tf.imsave(zstack_fn.parent / f'{zstack_fn.stem}_ch{ch}_reg_by_ch1.tif', zstack_ch_reg)


In [12]:
# it took 100 minutes to run both channels
# but ch0 should be registered using ch1 as reference
for ch in range(2):
    zstack_ch = np.squeeze(zstack[:,ch,:,:])
    zstack_ch_reg = get_zstack_reg(zstack_ch, num_slices, 60)
    tf.imsave(zstack_fn.parent / f'{zstack_fn.stem}_ch{ch}_reg.tif', zstack_ch_reg)

C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_30496\1218068999.py:4: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tf.imsave(zstack_fn.parent / f'{zstack_fn.stem}_ch{ch}_reg.tif', zstack_ch_reg)
